In [2]:
# Inspect data and plot figures
import numpy as np

import matplotlib.pyplot as plt
plt.style.use("seaborn-poster")
plt.rcParams["figure.figsize"] = (10, 6)

import pandas as pd

# Paths to data
base = "../data/base.csv"
opt = "../data/opt.csv"

# Load data
base = pd.read_csv(base, header=None)
opt = pd.read_csv(opt, header=None)
header = ["SIZE", "NODES", "CPUS_PER_NODE", "NBODIES", "RTIME", "GLOPS"]
base.set_axis(header, axis=1, inplace=True)
opt.set_axis(header, axis=1, inplace=True)

In [3]:
# Functions
def speedup(serial_rtime: float, parallel_rtime: float) -> float:
    return serial_rtime/parallel_rtime

def efficiency(speedup_val: float, n_procs: int) -> float:
    return speedup_val/n_procs

# Describe
## Initial Observations
* Min and Max of Optimized rtime is lower
* Mean of optimized program is actually lower
* How does scalability compare (i.e., as N increases, how do the strategies compare w.r.t rtime)?

## What Should Figures Show
* Speedup = serial rtime / parallel rtime # @ n-processes
* Efficiency = Speedup / n-processes
* Hard vs. Weak Scaling
* Serial Performance

In [4]:
base.describe()

,SIZE,NODES,CPUS_PER_NODE,NBODIES,RTIME,GLOPS
count,144.000000,144.000000,144.00000,144.000000,144.000000,144.000000
mean,35.000000,4.402778,6.75000,3908.000000,35.115382,1.761667
std,38.929738,2.348395,6.67057,3788.029434,84.855694,1.966040
min,2.000000,2.000000,1.00000,512.000000,0.180000,0.170000
25%,7.500000,2.000000,1.00000,896.000000,0.664500,0.520000
50%,20.000000,4.000000,4.00000,2560.000000,2.864500,0.955000
75%,40.000000,6.000000,16.00000,5572.000000,24.700000,2.260000
max,128.000000,8.000000,16.00000,10000.000000,505.927000,9.080000


In [13]:
base[(base["NBODIES"] == 10000) & (base["SIZE"] == 2)]["RTIME"]

36     504.297
84     503.987
132    505.927
Name: RTIME, dtype: float64

In [5]:
opt.describe()

,SIZE,NODES,CPUS_PER_NODE,NBODIES,RTIME,GLOPS
count,144.000000,144.000000,144.000000,144.000000,144.000000,144.000000
mean,35.000000,3.791667,3.750000,3908.000000,19.378958,2.183819
std,38.929738,2.058158,5.444571,3788.029434,52.185408,2.285562
min,2.000000,2.000000,1.000000,512.000000,0.103000,0.020000
25%,7.500000,2.000000,1.000000,896.000000,2.960500,0.205000
50%,20.000000,4.000000,1.000000,2560.000000,12.013500,1.470000
75%,40.000000,4.000000,4.000000,5572.000000,16.472000,3.572500
max,128.000000,8.000000,16.000000,10000.000000,366.462000,9.340000


# Summary Statistics

In [11]:
NBODIES = base["NBODIES"].unique()
SIZE = sorted(base["SIZE"].unique())
print(NBODIES)
print(SIZE)

base_mean_std = dict({"SIZE": [], "NBODIES": [], "MEAN_RTIME": [], "STD_RTIME": []})
opt_mean_std = dict({"SIZE": [], "NBODIES": [], "MEAN_RTIME": [], "STD_RTIME": []})

for nbody in NBODIES:
    for size in SIZE:
        base_duplicated: pd.DataFrame = base[(base["NBODIES"] == nbody) & (base["SIZE"] == size)]
        opt_duplicated: pd.DataFrame = opt[(opt["NBODIES"] == nbody) & (opt["SIZE"] == size)]

        base_mean = base_duplicated["RTIME"].mean()
        base_std = base_duplicated["RTIME"].mean()
        opt_mean = opt["RTIME"].mean()
        opt_std = opt["RTIME"].std()

        base_mean_std["SIZE"].append(size)
        base_mean_std["NBODIES"].append(nbody)
        base_mean_std["MEAN_RTIME"].append(base_mean)
        base_mean_std["STD_RTIME"].append(base_std)

        opt_mean_std["SIZE"].append(size )
        opt_mean_std["NBODIES"].append(nbody)
        opt_mean_std["MEAN_RTIME"].append(opt_mean)
        opt_mean_std["STD_RTIME"].append(opt_std)

[  512  1024  4096 10000]
[2, 4, 6, 8, 16, 24, 32, 64, 96, 128]
